# A Data Dive into Covid-19

Ralph Schlosser, 14/03/2020

Last updated: 01/04/2020

## References

* Data: Johns Hopkins CSSE GitHub repo: https://github.com/CSSEGISandData/COVID-19
* Forecasting with Facebook's Prophet: https://facebook.github.io/prophet/docs/quick_start.html#python-api



## Data loading and exploration

All the data gets loaded in CSV format from the above GitHub repository.

In [0]:
# Import relevant libs and tools.
import pandas as pd
import numpy as np
import logging
import plotly.graph_objects as go
from fbprophet import Prophet
from scipy.optimize import curve_fit
from datetime import date

In [0]:
# Set data URLs.
# Other possible source for Ireland: https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_the_Republic_of_Ireland

CONFIRMED_PATH = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
RECOVERED_PATH = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
DEATHS_PATH = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"

# Load raw data.
all_df = confirmed_df = pd.read_csv(CONFIRMED_PATH)
confirmed_df["status"] = "confirmed"

recovered_df = pd.read_csv(RECOVERED_PATH)
recovered_df["status"] = "recovered"

deaths_df = pd.read_csv(DEATHS_PATH)
deaths_df["status"] = "deceased"

# Put it in one dataframe.
all = [confirmed_df, recovered_df, deaths_df]
all_df = pd.concat(all)


In [0]:
def get_country_data(df, country="Spain", status="confirmed"):
  """
  Returns data rows for a particular country.
  """
  # Room for improvement. ;)
  country_level = df[df["Country/Region"] == country]
  country_by_status = country_level[country_level["status"] == status]

  return country_by_status.drop("status", axis=1)

In [0]:
# Pull per-country data.

china_confirmed = get_country_data(all_df, country="China")
italy_confirmed = get_country_data(all_df, country="Italy")
us_confirmed = get_country_data(all_df, country="US")
ireland_confirmed = get_country_data(all_df, country="Ireland")
germany_confirmed = get_country_data(all_df, country="Germany")
panama_confirmed = get_country_data(all_df, country="Panama")
spain_confirmed = get_country_data(all_df)

In [0]:
# Note: China has data at per-province level. So we need to aggregate things a bit.
# FIXME: Exclude long / lat as it doesn't make sense int this context.
china_confirmed_total = china_confirmed.groupby(by="Country/Region").sum().head()

In [0]:
def get_time_series(df, start_from = 4):
  """
  Cleans up the df and returns data as a time series, suitable for plotting and forecasting.
  FIXME: Check for NaN, etc.
  """
  df_ts = df.transpose()[start_from:]
  df_ts = df_ts.reset_index(drop=False, inplace=False)
  df_ts.columns=["date", "count"]
  df_ts["date"] = pd.to_datetime(df_ts["date"])
  return(df_ts)

In [0]:
# Get data in shape for TS plotting and analysis.
italy_confirmed_ts = get_time_series(italy_confirmed)
us_confirmed_ts = get_time_series(us_confirmed)
ireland_confirmed_ts = get_time_series(ireland_confirmed)
germany_confirmed_ts = get_time_series(germany_confirmed)
spain_confirmed_ts = get_time_series(spain_confirmed)
china_confirmed_ts = get_time_series(china_confirmed_total, start_from=2)

### Plot of confirmed case

In [8]:
# Create a plot with range slider.

today = date.today()

fig = go.Figure()
fig.add_trace(go.Scatter(x=italy_confirmed_ts.date, y=italy_confirmed_ts["count"], name="Italy confirmed",
                         line_color="deepskyblue"))

fig.add_trace(go.Scatter(x=spain_confirmed_ts.date, y=spain_confirmed_ts["count"], name="Spain confirmed",
                         line_color="goldenrod"))

fig.add_trace(go.Scatter(x=china_confirmed_ts.date, y=china_confirmed_ts["count"], name="China confirmed",
                         line_color="indianred"))

fig.add_trace(go.Scatter(x=ireland_confirmed_ts.date, y=ireland_confirmed_ts["count"], name="Ireland confirmed",
                         line_color="green"))

fig.add_trace(go.Scatter(x=germany_confirmed_ts.date, y=germany_confirmed_ts["count"], name="Germany confirmed",
                         line_color="darkslategrey"))

fig.add_trace(go.Scatter(x=us_confirmed_ts.date, y=us_confirmed_ts["count"], name="US confirmed",
                         line_color="darkmagenta"))

fig.update_layout(title_text=f"Covid-19 Confirmed Cases, {today}",
                  xaxis_rangeslider_visible=True)
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Number of cases",
    font=dict(
        family="Helvetica, monospace",
        size=12
    )
)
fig.show()

### Change points observations

These may be useful to consider in terms of improving the Time Series model.

* February 12th 2020 appears to be a change point for China. There are a few other spikes which may be to do with a change in reporting?
* March 12th seems to be a change point for Italy as well as Spain. Curious.



## Forecasting confirmed cases with Prophet

Now let's train time series models for each country with Prophet.

We will be running a 5 day forecast for each country. I'm not doing much tweaking here, so there is plenty of room for improvements.

At it's core, Prophet uses a type of regression model, similar to a GAM, to fit a three-component time series model, where the target $y(t)$ is expressed as a sum of three functions:

$$
y(t) = g(t) + s(t) + h(t) + \epsilon_t
$$

with:

* $g(t)$ - Trend function capturing non-periodic changes
* $s(t)$ - Trend function modelling periodic changes, i.e. seasonality component
* $h(t)$ - Models the effect of holidays
* $\epsilon_t$ - A noise term that reflects random, or non-systematic changes

The original prohpet paper goes into a lot more detail: https://peerj.com/preprints/3190.pdf

### China


In [9]:
# Mute logging, if needed.
# logging.getLogger().setLevel(logging.INFO)

def train_prohpet_model(df, chg_pts):
  """
  Train the model using Prophet.
  """
  # Need to change the columns, so it's compatible with Prophet.
  df.columns=["ds", "y"]
  model = Prophet(changepoints=chg_pts)
  model.fit(df)
  return model

def get_forecast(df, num_days, chg_pts):
  """
  Build model and return forecast for given number of days.
  """
  country_df = df.copy()
  country_model = train_prohpet_model(country_df, chg_pts)
  future = country_model.make_future_dataframe(periods=num_days)
  forecast = country_model.predict(future)
  return(country_model, forecast)

# FIXME: Set this to how many days you want.
DAYS = 5

cm, china_fc = get_forecast(df = china_confirmed_ts, 
                        num_days = DAYS,
                        chg_pts = None)

# Let's take a look into the future!
china_fc[["ds", "yhat"]].tail()

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,yhat
135,2020-06-05,85103.511478
136,2020-06-06,84923.644290
137,2020-06-07,84852.849289
138,2020-06-08,84661.789155
139,2020-06-09,84639.148883


In [10]:
# Plot data
from fbprophet.plot import plot_plotly
today = date.today()

def make_plot(model, forecast, days, country):
  """
  Utility function to create these plots for several countries.
  """
  fig = plot_plotly(model, forecast) 

  fig.update_layout(title_text=f"Confirmed Cases {country}, {today}, {days} days forecast",
                   xaxis_rangeslider_visible=False)
  fig.update_layout(
      xaxis_title="Date",
      yaxis_title="Number of cases",
      font=dict(
          family="Helvetica, monospace",
          size=12
      )
  )
  return(fig)

# Create the China plot.
china_fig = make_plot(cm, china_fc, DAYS, "China")
china_fig.show()

**Result**: The model's trendline appears to be following the data to some extent. There are some deviations from the data points, particularly for early and close to current times. For the latter, we are over-estimating the number of confirmed cases.

FIXME:

* Try adding change points
* Parameter tweaking
* Fix model diagnostics.

#### Model diagnostics

Something not right here, looks like the model isn't performing that well as the plot has us believe.

In [11]:
from fbprophet.diagnostics import cross_validation, performance_metrics

# FIXME: There is something not right here!!
def get_performance_metrics(model, horizon="10 days"):
  df_cv = cross_validation(model, horizon)
  return(performance_metrics(df_cv))


china_pm = get_performance_metrics(cm)
china_pm.head()

INFO:fbprophet:Making 19 forecasts with cutoffs between 2020-02-25 00:00:00 and 2020-05-25 00:00:00


,horizon,mse,rmse,mae,mape,mdape,coverage
0,1 days,2.772118e+07,5265.090618,2973.015539,0.036923,0.013582,0.789474
1,2 days,3.807372e+07,6170.390665,3364.775637,0.041770,0.012276,0.789474
2,3 days,4.747467e+07,6890.186775,3670.460697,0.045510,0.013676,0.789474
3,4 days,5.917853e+07,7692.758189,4047.034282,0.050064,0.015302,0.789474
4,5 days,6.962313e+07,8344.047697,4377.616650,0.054010,0.016232,0.789474


### Italy

In [12]:
im, italy_fc = get_forecast(df = italy_confirmed_ts, 
                        num_days = DAYS,
                        chg_pts = None)

italy_fc[["ds", "yhat"]].tail()

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,yhat
135,2020-06-05,236655.710805
136,2020-06-06,237473.589062
137,2020-06-07,238142.190437
138,2020-06-08,238502.491519
139,2020-06-09,238928.365303


In [13]:
italy_fig = make_plot(im, italy_fc, DAYS, "Italy")
italy_fig.show()

### Fitting an exponential model


This was an attempt to use an exponential model which may work better in the early stages of an outbreak.

$$
y(t) = \beta_2 e^{\beta_1 t} + \beta_0
$$

**FIXME**: Need to change the prediction so that we'll only forecast 2, 3 days into the future.


In [0]:
def exponential_model(t, beta2, beta1, beta0):
  """
  Proposed model function.
  """
  return beta2 * np.exp(beta1 * t) + beta0

def fit_and_make_plot(df, model_func, country, num_days = 20):
  """
  Fit the defined model and create a plot.
  """

  # Add a column for consecutive days.
  today = df.shape[0]
  start = today - num_days 
  xvals = pd.Series(range(start, today))

  # Fit the model function.
  popt, pcov = curve_fit(model_func, xvals, df["count"][start:today])

  fig = go.Figure()

  # Actual data points.
  fig.add_trace(go.Scatter(x=xvals, 
                           y=df["count"][start:today], 
                           name=f"{country} confirmed cases for the last {num_days} days",
                         line_color="deepskyblue"))

  # Predicted data points.
  # FIXME: Clearly we should only predict for 2, 3 days!!
  fig.add_trace(go.Scatter(x=xvals, 
                           y=model_func(xvals, *popt), name=f"{country} confirmed predicted for the last {num_days} days",
                        line_color="indianred"))



  fig.update_layout(title_text="Covid-19 Confirmed Cases Prediction",
                    xaxis_rangeslider_visible=False)
  
  fig.update_layout(
      xaxis_title="Date",
      yaxis_title="Number of cases",
      font=dict(
          family="Helvetica, monospace",
          size=12
      )
  )
  return(fig)

In [16]:
fig_italy = fit_and_make_plot(italy_confirmed_ts, exponential_model, "Italy", num_days=35)

fig_italy.show()

/usr/local/lib/python3.6/dist-packages/scipy/optimize/minpack.py:808: OptimizeWarning:

Covariance of the parameters could not be estimated



### Spain

In [17]:
sm, spain_fc = get_forecast(df = spain_confirmed_ts, 
                        num_days = DAYS,
                        chg_pts = None)

spain_fc[["ds", "yhat"]].tail()

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,yhat
135,2020-06-05,243274.775570
136,2020-06-06,243886.098620
137,2020-06-07,244556.887259
138,2020-06-08,245085.086813
139,2020-06-09,245537.848951


In [18]:
spain_fig = make_plot(im, italy_fc, DAYS, "Spain")
spain_fig.show()

## SIR model parameter estimation

### Estimating $R_0$ for Ireland

Good references: 

* https://kingaa.github.io/clim-dis/parest/parest.html#the-sir-model
* http://faculty.olin.edu/bstorey/Notes/DiffEq.pdf


**TODO**

### Numerically solving ODEs

In [19]:
# SIR models involve solving ODEs numerically.
# Let's do some tests in Python first.

from scipy.integrate import odeint

def model(y, t, k):
    dydt = -k * y
    return dydt

# Initial condition
y0 = 5

# Time points
t = np.linspace(0,20)

# Solve ODEs
k = 0.1
y1 = odeint(model,y0,t,args=(k,))
k = 0.2
y2 = odeint(model,y0,t,args=(k,))
k = 0.5
y3 = odeint(model,y0,t,args=(k,))

print(y1)

[[5.        ]
 [4.80002727]
 [4.60805231]
 [4.42375528]
 [4.24682914]
 [4.07697907]
 [3.91392204]
 [3.75738646]
 [3.60711151]
 [3.46284668]
 [3.32435164]
 [3.19139567]
 [3.06375722]
 [2.9412236 ]
 [2.82359066]
 [2.7106624 ]
 [2.60225066]
 [2.49817479]
 [2.3982614 ]
 [2.30234402]
 [2.21026283]
 [2.12186437]
 [2.03700131]
 [1.95553229]
 [1.87732164]
 [1.80223902]
 [1.73015927]
 [1.6609623 ]
 [1.59453284]
 [1.53076021]
 [1.46953813]
 [1.4107646 ]
 [1.35434169]
 [1.30017539]
 [1.24817545]
 [1.19825522]
 [1.15033154]
 [1.10432453]
 [1.06015756]
 [1.01775702]
 [0.97705227]
 [0.93797549]
 [0.90046157]
 [0.86444801]
 [0.82987478]
 [0.79668431]
 [0.76482127]
 [0.73423257]
 [0.70486726]
 [0.6766764 ]]
